<a href="https://colab.research.google.com/github/patrick-anastasio/dsc-resilient-distributed-datasets-rdd-lab/blob/master/completed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="https://github.com/patrick-anastasio/dsc-resilient-distributed-datasets-rdd-lab/blob/master/images/rdd_diagram.png?raw=1" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = range(1, 1001)
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [3]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 55.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=1c24dd11d3d994c3d0819af621d02da7f2117d7a1ecf9b9ad62ad9a09338bc4f
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [6]:
rdd = sc.parallelize(data, 10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [7]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [8]:
# count
rdd.count()

1000

In [9]:
# first
rdd.first()

1

In [10]:
# take
rdd.take(5)

[1, 2, 3, 4, 5]

In [11]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [13]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [14]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([6.57013908e-01, 7.51030427e-01, 1.31829312e+00, 5.30077579e-01,
       4.54004576e+00, 2.56815305e+00, 5.17041341e+00, 2.30270384e+00,
       2.26463488e+00, 8.53715383e+00, 3.91778241e+00, 4.12576569e+00,
       6.78910395e+00, 4.03576140e+00, 1.83138429e+00, 9.35841289e+00,
       1.11600840e+01, 9.67293660e+00, 6.84511470e+00, 4.34341647e+00,
       1.50099829e+01, 7.59884833e+00, 6.22173440e+00, 1.06090084e+01,
       1.72704194e+01, 1.60323856e+01, 4.26893888e+00, 2.45781842e+01,
       2.85127966e+01, 2.16478945e+01, 2.94212562e+01, 1.10146874e+01,
       1.66861615e+01, 7.34505402e+00, 1.58387747e+00, 6.93555257e-01,
       1.11552996e+01, 2.98132214e+01, 1.26628473e+01, 3.11588283e+01,
       8.16763204e+00, 7.04730724e+00, 1.69369817e+01, 1.03578350e+01,
       1.06536045e+00, 4.42894783e+01, 1.03642801e+01, 2.34624073e+01,
       4.51857994e+01, 4.40920489e+01, 1.07978419e+01, 2.07403089e+01,
       3.95755611e+01, 6.43554656e+00, 2.50453305e+01, 1.79086077e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [16]:
price_items = sc.parallelize(sales_figures, 10)
price_items.take(5)

[0.6570139079953231,
 0.7510304273307942,
 1.3182931209011572,
 0.5300775794809081,
 4.540045756288523]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [17]:
def sales_tax(num):
  return num*.98

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [18]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.6438736298354166,
 0.7360098187841783,
 1.291927258483134,
 0.5194760278912899,
 4.449244841162752,
 2.51678998796202,
 5.067005145653943,
 2.2566497622433865,
 2.219342181986122,
 8.366410756007788]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [19]:
discounted = revenue_minus_tax.map(lambda x : x*0.9)

In [20]:
discounted.take(10)

[0.579486266851875,
 0.6624088369057605,
 1.1627345326348206,
 0.4675284251021609,
 4.004320357046477,
 2.265110989165818,
 4.560304631088549,
 2.0309847860190477,
 1.9974079637875097,
 7.52976968040701]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [23]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[838.6143842729786,
 824.7275991165474,
 808.4892360663181,
 805.8690712961411,
 800.6306286287944,
 799.7793004875346,
 798.4940798685169,
 791.6534214372964,
 785.3368752714672,
 779.0732084376859,
 773.3915580215969,
 771.4762662136534,
 766.938220635069,
 765.8817844525906,
 760.5075092520711]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [26]:
discounted.toDebugString()

b'(10) PythonRDD[14] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[7] at readRDDFromFile at PythonRDD.scala:274 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [27]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.6570139079953231, 0.5440075158201276), (0.7510304273307942, 0.6218531938298976), (1.3182931209011572, 1.0915467041061582), (0.5300775794809081, 0.4389042358101919), (4.540045756288523, 3.759157886206897), (2.568153048940837, 2.126430724523013), (5.170413413932595, 4.281102306736188), (2.302703839023864, 1.9066387787117596), (2.2646348795776756, 1.8751176802903153), (8.537153832661009, 7.068763373443316)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [28]:
flat_mapped = price_items.flatMap(lambda x : (x, x*0.92*0.9 ))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.6570139079953231, 0.5440075158201276, 0.7510304273307942, 0.6218531938298976, 1.3182931209011572, 1.0915467041061582, 0.5300775794809081, 0.4389042358101919, 4.540045756288523, 3.759157886206897]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [32]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)

# calculate total remaining in inventory 
selected_items.count()

322

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [33]:
selected_items.reduce(lambda x,y: x + y)

148374.93458355256

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [34]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(43, 304.4306498235886),
 (36, 307.94233066435226),
 (25, 301.03545854692373),
 (18, 329.98973416739295),
 (25, 324.52582601532185),
 (35, 334.2787528195129),
 (19, 316.28771622382084)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [36]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)

In [37]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(32, 5803.439863780104),
 (50, 5803.263700378549),
 (9, 5299.98156039519),
 (30, 4903.007182342491),
 (16, 4802.338358576729),
 (28, 4504.351605936608),
 (31, 4342.04506687522),
 (37, 4276.674987965187),
 (25, 3973.2962805951647),
 (7, 3858.497560450051),
 (22, 3784.007803002143),
 (49, 3679.112925826701),
 (21, 3626.4229943860096),
 (12, 3605.3537485777733),
 (38, 3594.6107741115356),
 (42, 3541.1275935187514),
 (15, 3228.739888550137),
 (35, 3221.036241161615),
 (36, 3105.2911651602867),
 (11, 3041.666202069363),
 (4, 3026.354621526395),
 (24, 3007.3214859356017),
 (6, 2956.746368845522),
 (44, 2942.13266842338),
 (40, 2851.929221127071),
 (3, 2848.103640718851),
 (23, 2785.9063720272316),
 (46, 2702.6813138350353),
 (1, 2655.1430052263963),
 (34, 2581.5089079580575),
 (41, 2542.4934541129774),
 (5, 2539.5897071029785),
 (27, 2465.443760735251),
 (20, 2460.312514715358),
 (14, 2319.0004487798587),
 (39, 2288.5884747132254),
 (18, 2272.5797225783936),
 (2, 2228.329274304245),
 (13, 2

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [42]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(3, 11),
 (40, 11),
 (37, 10),
 (32, 10),
 (33, 10),
 (9, 9),
 (22, 9),
 (42, 9),
 (34, 8),
 (18, 8),
 (39, 8),
 (41, 8),
 (46, 8),
 (20, 8),
 (45, 8),
 (29, 7),
 (5, 7),
 (14, 7),
 (12, 7),
 (7, 7),
 (11, 7),
 (28, 7),
 (49, 7),
 (10, 7),
 (21, 7),
 (31, 7),
 (36, 7),
 (50, 6),
 (38, 6),
 (48, 6),
 (15, 6),
 (26, 6),
 (44, 5),
 (16, 5),
 (2, 5),
 (13, 5),
 (24, 5),
 (43, 5),
 (17, 5),
 (8, 5),
 (23, 4),
 (19, 4),
 (47, 4),
 (25, 4),
 (35, 4),
 (1, 3),
 (27, 3),
 (4, 3),
 (6, 2),
 (30, 2)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
